# Proyek: Analisis Prediksi Biaya Medis
## Notebook 01: Data Preparation

**Fase CRISP-ML(Q):** 3. Data Preparation

**Tujuan:**
1.  Memuat data mentah dari `data/raw/`.
2.  Mem-prototipe (menguji coba) langkah-langkah *preprocessing* (One-Hot Encoding, Scaling) yang diidentifikasi selama EDA (Notebook 00).
3.  Memvalidasi secara visual bahwa langkah-langkah tersebut berfungsi seperti yang diharapkan.
4.  Mendesain arsitektur `Pipeline` dan `ColumnTransformer` yang akan digunakan dalam *notebook* pemodelan.
5.  (Opsional) Menyimpan *snapshot* data yang telah diproses ke `data/prepared/` **hanya untuk tujuan inspeksi manual**, bukan untuk pemodelan akhir.

## 1. Setup & Pemuatan Data

In [1]:
import pandas as pd
import numpy as np
import os

# Definisikan path kita
RAW_DATA_PATH = '../data/raw/insurance.csv'
PREPARED_DATA_DIR = '../data/prepared/'
PREPARED_DATA_PATH = os.path.join(PREPARED_DATA_DIR, 'insurance_features.csv')

print("Paths defined.")

Paths defined.


In [2]:
try:
    df = pd.read_csv(RAW_DATA_PATH)
    print(f"Data mentah loaded successfully from {RAW_DATA_PATH}")
    print(f"Original shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {RAW_DATA_PATH}")

Data mentah loaded successfully from ../data/raw/insurance.csv
Original shape: (1338, 7)


## 2. Feature Engineering

Berdasarkan EDA (Notebook 00), kita memiliki dua tugas utama:
1.  **Transformasi Target:** Membuat `log_charges` untuk membantu model linear.
2.  **Encoding Kategorikal:** Mengubah `sex`, `smoker`, dan `region` menjadi numerik.

### 2.1 Transformasi Variabel Target (`charges`)

Kita akan membuat kolom `log_charges` baru menggunakan `np.log1p` (log(1+x)) untuk menangani distribusi yang miring (skewed).

In [3]:
df_processed = df.copy()

df_processed['log_charges'] = np.log1p(df_processed['charges'])

print("Kolom 'log_charges' berhasil dibuat.")
print(df_processed[['charges', 'log_charges']].head())

Kolom 'log_charges' berhasil dibuat.
       charges  log_charges
0  16884.92400     9.734236
1   1725.55230     7.453882
2   4449.46200     8.400763
3  21984.47061     9.998137
4   3866.85520     8.260455


### 2.2 Encoding Variabel Kategorikal

Kita akan menggunakan **One-Hot Encoding** (`pd.get_dummies`) untuk mengubah `sex`, `smoker`, dan `region`.

Kita akan menggunakan `drop_first=True` untuk menghindari *multicollinearity* (Dummy Variable Trap). Ini adalah praktik terbaik, terutama untuk model linear (GLM, Logistic Regression) yang akan kita gunakan nanti.

In [4]:
categorical_cols = ['sex', 'smoker', 'region']

df_processed = pd.get_dummies(df_processed, 
                              columns=categorical_cols, 
                              drop_first=True, 
                              dtype=int) # Konversi dummy ke integer (0/1)

print("One-Hot Encoding selesai.")

One-Hot Encoding selesai.


## 3. Finalisasi & Penyimpanan

Kita akan meninjau DataFrame akhir kita dan menyimpannya ke `data/prepared/`.

In [5]:
print("--- Final DataFrame Head ---")
print(df_processed.head())

print("\n--- Final DataFrame Info ---")
df_processed.info()

print(f"\nBentuk baru setelah OHE: {df_processed.shape}")

--- Final DataFrame Head ---
   age     bmi  children      charges  log_charges  sex_male  smoker_yes  \
0   19  27.900         0  16884.92400     9.734236         0           1   
1   18  33.770         1   1725.55230     7.453882         1           0   
2   28  33.000         3   4449.46200     8.400763         1           0   
3   33  22.705         0  21984.47061     9.998137         1           0   
4   32  28.880         0   3866.85520     8.260455         1           0   

   region_northwest  region_southeast  region_southwest  
0                 0                 0                 1  
1                 0                 1                 0  
2                 0                 1                 0  
3                 1                 0                 0  
4                 1                 0                 0  

--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Nu

In [6]:
# Pastikan direktori 'prepared' ada
os.makedirs(PREPARED_DATA_DIR, exist_ok=True)

# Simpan ke CSV
# Kita gunakan index=False karena kita tidak ingin menyimpan index pandas ke file
try:
    df_processed.to_csv(PREPARED_DATA_PATH, index=False)
    print(f"\nData yang sudah diproses berhasil disimpan ke: {PREPARED_DATA_PATH}")
except Exception as e:
    print(f"\nError saat menyimpan file: {e}")


Data yang sudah diproses berhasil disimpan ke: ../data/prepared/insurance_features.csv


## Kesimpulan Fase 3

Kita telah berhasil mengubah data mentah menjadi `insurance_features.csv` yang bersih.

**Perubahan yang Dilakukan:**
1.  Kolom `log_charges` ditambahkan.
2.  `sex` diubah menjadi `sex_male` (0 atau 1).
3.  `smoker` diubah menjadi `smoker_yes` (0 atau 1).
4.  `region` diubah menjadi `region_northwest`, `region_southeast`, `region_southwest`.

Data ini sekarang siap untuk **Langkah 02: Modeling Cost Regression**.